<a href="https://colab.research.google.com/github/dtabuena/Patch_Ephys/blob/main/IV_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def IV_analyzer(abf,measure_windows={'IV_Early':(16, 30),'IV_Steady_State':(100,120)},to_plot=False,figopt={'type':'jpg','dpi':300}):
    'Takes the abf data and calculates the mean and peak for each window.'
    'Optionaly sweeps with APs can be filtered out using spike_threshold.'
    'Only sweeps passing the QC filter are used. Returns a list of dicts with'
    'each dict corresponding to one anaysis window reporting the time range, peak etc.'

    # IV_stats = []
    # theta, offset, correct_ch1 = predict_telegraph(abf)

    results = {}


    if to_plot:
        fig, axs = plt.subplots(len(measure_windows),2,figsize = [4.5,3] )
        # if len(measure_windows.keys())==1: axs = [axs]

    for w_i in range(len(measure_windows.keys())):
        w = list(measure_windows.keys())[w_i]
        window_stats = {}
        start = measure_windows[w][0]/1000 # ms to s
        stop = measure_windows[w][1]/1000
        delta_t = stop-start
        flank_neg = start - (delta_t)*0.5
        flank_pos = stop + (delta_t)*0.5
        full_plot_range_ind =   np.logical_and(  abf.sweepX>flank_neg, abf.sweepX<flank_pos )
        spikeless_sweeps = []
        # average
        for s in abf.sweepList:
            if to_plot: s_shift_t = delta_t*0.03*s
            if to_plot: s_shift_I = 100*s
            abf.setSweep(s)
            if abs(abf.sweepY[-1])>1000: continue # cell blew up, bail
            time_abrg = abf.sweepX[full_plot_range_ind]
            I_abrg = abf.sweepY[full_plot_range_ind]

            time_abrg = abf.sweepX[full_plot_range_ind]
            I_abrg = abf.sweepY[full_plot_range_ind]
            analysis_range = np.logical_and(  abf.sweepX>start, abf.sweepX<stop )
            time_analysis = abf.sweepX[analysis_range]
            I_analysis = movmean( abf.sweepY[analysis_range], 4)
            dIdt = movmean(np.diff(I_abrg, prepend=I_abrg[0]),1)


        I_peak_l = []
        I_mean_l = []
        V_stim_l = []

        # measured v_hold
        abf.setSweep(sweepNumber=0, channel=1)
        v_hold = round(abf.sweepY[0]*10)/10
        abf.setSweep(sweepNumber=0, channel=0)

        # peak
        for s in abf.sweepList :
            abf.setSweep(s)
            if abs(abf.sweepY[-1])>1000: continue # cell blew up, bail
            s_shift_t = delta_t*0.03*s * 0
            s_shift_I = 100*s * 0

            time_abrg = abf.sweepX[full_plot_range_ind]
            I_abrg = abf.sweepY[full_plot_range_ind]
            analysis_range = np.logical_and(  abf.sweepX>start, abf.sweepX<stop )
            time_analysis = abf.sweepX[analysis_range]
            I_analysis = abf.sweepY[analysis_range]


            # Peaks
            if to_plot: axs[w_i,0].plot( time_abrg+s_shift_t, I_abrg+s_shift_I,'k')

            I_mean = np.median( I_analysis )
            if to_plot: axs[w_i,0].plot( time_analysis+s_shift_t, (I_mean*np.ones_like(time_analysis))+s_shift_I,'r' )

            local_I_mean = movmean(I_analysis,100)
            delta_i = I_analysis #-I_mean
            peak_ind = np.where(abs(delta_i) == np.max(abs(delta_i)))[0][0]

            I_peak = delta_i[peak_ind]
            t_peak = time_analysis[peak_ind]
            if to_plot: axs[w_i,0].scatter( t_peak+s_shift_t, I_peak, color='m' )

            command_trace = abf.sweepC + v_hold
            # print('vh, cmd',v_hold, command_trace[analysis_range][-1] )

            I_peak_l.append(I_peak)
            I_mean_l.append(I_mean)

            command_v = round(np.median( command_trace[analysis_range])/10)*10
            V_stim_l.append(command_v)

        # print('I_peak_l', I_peak_l)
        # print('I_mean_l', I_mean_l)
        # print('V_stim_l', V_stim_l)

        window_stats['range'] = (start*1000,stop*1000)
        window_stats['I_peak'] = I_peak_l
        window_stats['I_mean'] = I_mean_l
        window_stats['V_stim'] = V_stim_l


        results[w] = window_stats
        if to_plot:
            axs[w_i,1].plot(window_stats['V_stim'],window_stats['I_peak'] , '-o',color='magenta'  )
            axs[w_i,1].plot(window_stats['V_stim'],window_stats['I_mean'] , '-o',color='red')
            axs[w_i,1].spines['left'].set_position('zero')
            axs[w_i,1].spines['bottom'].set_position('zero')

    # plt.show()
    if to_plot:
        try:    os.makedirs('Saved_Figs/IV_Curves/')
        except:     None
        plt.savefig( 'Saved_Figs/IV_Curves/IV_Curves'+'_' + abf.abfID +'.'+figopt['type'],dpi=figopt['dpi'])
        plt.show()

    return results


# fn = 'my_ephys_data/2022_08_12_RNF182/2022x08x12_RNF182_E4KI_F_P251_s001_c002_CA3xPOS_0002.abf'
# IV_analyzer
# results = IV_analyzer(fn,to_plot=True)

# # print(results)

In [ ]:

def IV_analyzer_v2(abf,measure_windows={'IV_Early':(16, 30),'IV_Steady_State':(100,120)},to_plot=[False, False],figopt={'type':'jpg','dpi':300},pn=True,use_comp=True):
    if len(abf.sweepList)<7:
        return {}
    sweep_df = pseudo_pn_subtraction(abf,to_plot=to_plot[0])
    results = iv_analysis_df(sweep_df,abf,measure_windows=measure_windows,to_plot=to_plot[1],pn=pn,figopt=figopt)
    is_compensated, sum_delta = check_wc_comp(abf)
    results['is_compensated'] = is_compensated
    results['sum_delta'] = sum_delta
    if is_compensated == use_comp: return results
    else: return {}
    return results


def pseudo_pn_subtraction(abf,to_plot=False):
    'Uses one of the low pulses to estimate P/N subtraction'

    sweep_df = pd.DataFrame({'sweep':abf.sweepList}).set_index('sweep')
    sweep_df['sweepX'] = None
    sweep_df['sweepY'] = None
    sweep_df['sweepY_pn'] = None
    sweep_df['sweepC'] = None
    sweep_df['stim'] = None
    sweep_df['N'] = None

    # measured v_hold
    abf.setSweep(sweepNumber=0, channel=1)
    v_hold = round(abf.sweepY[0]/10)*10
    abf.setSweep(sweepNumber=0, channel=0)


    for s in abf.sweepList:
        abf.setSweep(s)
        sweep_df.at[s,'sweepX'] = abf.sweepX
        sweep_df.at[s,'sweepY'] = abf.sweepY
        sweep_df.at[s,'sweepC'] = abf.sweepC
        sweep_df.at[s,'stim'] = max(abf.sweepC.min(), abf.sweepC.max(), key=abs)
        sweep_df.at[s,'Vm'] = sweep_df.at[s,'stim'] + v_hold

    stim_list = list(sweep_df['stim'])
    delta = np.min([abs(v) for v in stim_list if not v==0])

    if len(abf.sweepList)<7:
        return sweep_df

    sweep_df['N'] = sweep_df['stim']/delta
    is_p = list(abs(sweep_df['N'])==1)
    p_list = []
    for i in range(len(is_p)):
        if is_p[i]:
            p_trace =  sweep_df.loc[i,'sweepY'] * sweep_df.loc[i,'N']
            p_trace = p_trace - p_trace[0]
            p_list.append(  p_trace )
    p_val = np.mean(  np.stack(p_list, axis=0)  , axis=0 )
    if to_plot: fig, ax = plt.subplots(len(abf.sweepList),figsize=(2.5, 50))
    x_lim = [.015, 0.02]
    y_lim = [ -20000, 10000]
    for s in abf.sweepList:
        correction = (p_val * sweep_df.loc[s,'N'])
        sweep_df.at[s,'sweepY_pn'] = sweep_df.at[s,'sweepY'] - correction
        sweep_df.at[s,'sweepY_pn'] = sweep_df.at[s,'sweepY_pn'] - sweep_df.at[s,'sweepY_pn'][0]
        if to_plot:
            ax[s].plot(sweep_df.loc[s,'sweepX'],sweep_df.loc[s,'sweepY'],'k')
            ax[s].plot(sweep_df.loc[s,'sweepX'],sweep_df.loc[s,'sweepY_pn'],'m')
            ax[s].plot(sweep_df.loc[s,'sweepX'],correction,'c')
            ax[s].set_xlim(*x_lim)
            ax[s].set_title( sweep_df.loc[s,'stim'] )
    if to_plot: plt.show()

    return sweep_df



def iv_analysis_df(sweep_df, abf,measure_windows={'IV_Early':(16, 35),'IV_Steady_State':(100,120)},to_plot=False,pn=True,figopt={'type':'jpg','dpi':300}):
    results = {}
    if pn: y_col = 'sweepY_pn'
    else:  y_col = 'sweepY'
    if to_plot:
        fig, axs = plt.subplots(2,len(measure_windows),figsize = [4.5,3] )
    for w_i in range(len(measure_windows.keys())):
        w = list(measure_windows.keys())[w_i]
        w_range_s = np.array(measure_windows[w])/1000
        plot_range = (np.diff(w_range_s) * np.array([-1, 1]) * .5) + w_range_s
        in_window_bool = np.logical_and(sweep_df.loc[0,'sweepX'] >= w_range_s[0] ,  sweep_df.loc[0,'sweepX'] <= w_range_s[1])
        in_window_i = [int(i) for i in range(len(in_window_bool)) if in_window_bool[i]]

        # max
        # print(plot_range)
        peak_list = []
        median_list = []
        stim_list = []

        for s in sweep_df.index:
            if abs(sweep_df.loc[s,y_col][-1])>1000:
                'bad sweep, bail'
                continue
            oscl_check = np.max(abs(sweep_df.loc[s,y_col]))>10000
            if oscl_check:
                'bad sweep, bail'
                continue

            neg_peak = min(sweep_df.loc[s,y_col][in_window_i])
            wind_median = np.median(sweep_df.loc[s,y_col][in_window_i])
            peak_ind = np.where(sweep_df.loc[s,y_col] == neg_peak )[0][0]
            if to_plot:
                axs[0,w_i].plot(sweep_df.loc[s,'sweepX'],sweep_df.loc[s,y_col],'k')
                axs[0,w_i].set_xlim(*plot_range)
                axs[0,w_i].scatter(sweep_df.loc[s,'sweepX'][peak_ind],sweep_df.loc[s,y_col][peak_ind],color='m')
                med_line = wind_median*np.ones_like( in_window_i )
                axs[0,w_i].plot( sweep_df.loc[s,'sweepX'][in_window_i], med_line,'turquoise')

            peak_list.append(neg_peak)
            median_list.append(wind_median)
            stim_list.append(sweep_df.loc[s,'Vm']) # iterate the list to garantee matching stims and responses

        results[w] = {'range':measure_windows[w],
                        'I_peak':peak_list,
                        'I_mean':median_list,
                        'V_stim':stim_list}

        if to_plot:
            axs[1,w_i].spines['left'].set_position('zero')
            axs[1,w_i].spines['bottom'].set_position('zero')
            axs[1,w_i].plot(stim_list,peak_list,'-o',color='m')
            axs[1,w_i].plot(stim_list,median_list,'-o',color='turquoise')


    if to_plot:
        try:    os.makedirs('Saved_Figs/IV_Curves/')
        except:     None
        plt.savefig( 'Saved_Figs/IV_Curves/IV_Curves'+'_' + abf.abfID +'.'+figopt['type'],dpi=figopt['dpi'])
        plt.show()
    return results




def check_wc_comp(abf,cut_off=40,verbose =False,to_plot=False):
    sweep_df = pseudo_pn_subtraction(abf)
    sweep_df['command_real']=None
    is_compensated = 'unknown'
    is_hold_ind = [ i for i in range(len(sweep_df.at[0,'sweepC'])) if  sweep_df.at[0,'sweepC'][i]==sweep_df.at[0,'sweepC'][0]]
    is_stim =  1*np.logical_not(sweep_df.at[0,'sweepC']==sweep_df.at[0,'sweepC'][0])
    offset_ind = np.where( np.diff( is_stim, prepend = is_stim[0]) == -1)[0][0]
    window_ms = [-0.5, 1]
    window_I_range = [int(w*abf.sampleRate/1000) for w in window_ms]
    window_I = np.arange(*window_I_range)
    # print(window_I)
    offset_range = list(offset_ind + window_I)
    s = 0
    abf.setSweep(sweepNumber=s, channel=1)
    sweep_df.at[s,'command_real'] = abf.sweepY
    command_real_bal = sweep_df.at[s,'command_real'][offset_range] - sweep_df.at[s,'command_real'][0]
    sweepC = sweep_df.at[s,'sweepC'][offset_range]
    sum_delta = np.sum(abs(command_real_bal- sweepC))

    if to_plot:
        plt.plot( sweep_df.at[s,'command_real'][offset_range] - sweep_df.at[s,'command_real'][0] )
        plt.plot( sweep_df.at[s,'sweepC'][offset_range] )
        plt.show()

    # sum_delta = np.sum((sweep_df.at[0,'command_real'][offset_range]-sweep_df.at[0,'command_real'][0]) - sweep_df.at[0,'sweepC'][offset_range])
    if verbose: print(sum_delta)
    is_compensated = sum_delta>cut_off
    return is_compensated, sum_delta


# abf = abf_or_name('my_ephys_data/2022_08_15/2022x08x15_RNF182_E4KI_F_P251_s001_c001_CA3xPOS_0002.abf')
# _ = IV_analyzer_v2(abf)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

def movmean(x, n=3):
    """Simple moving-average function to smooth the signal."""
    return np.convolve(x, np.ones(n)/n, mode='same')

def IV_analyzer_v3(
    abf,
    Na_window=(16, 30),    # ms for Na window
    K_window=(100, 120),   # ms for K window
    to_plot=True,
    figopt={'type': 'jpg', 'dpi': 300}
):
    """
    1) Measures Na-peak (raw) in Na_window.
    2) Measures K-mean (raw) in K_window.
    3) Plots three subplots (Na-window, K full trace, IV).
       - Na-window trace is restricted to the window ± some margin.
       - K subplot is the entire trace, with a marker for the mean.
       - IV subplot has no legend and includes lines at x=0, y=0.

    Returns:
        results = {
            f"Na_{Na_window}": { voltage: raw_peak, ... },
            f"K_{K_window}":   { voltage: raw_mean, ... }
        }
    """

    # Convert ms to seconds for the analysis windows
    na_start_s = Na_window[0] / 1000
    na_stop_s = Na_window[1] / 1000
    k_start_s = K_window[0] / 1000
    k_stop_s = K_window[1] / 1000

    # Calculate durations
    na_delta_s = na_stop_s - na_start_s
    k_delta_s = k_stop_s - k_start_s

    # Prepare data containers
    na_peaks = []
    na_voltages = []
    k_means = []
    k_voltages = []

    # ---- Create figure with 3 subplots: (Na-window, K full, IV) ----
    if to_plot:
        fig, (ax_na, ax_k, ax_iv) = plt.subplots(1, 3, figsize=(6, 2.5))  # total figure size
    else:
        # Dummy axes if not plotting
        ax_na = ax_k = ax_iv = None

    # ----------------- NA MEASUREMENT & PLOT (SUBPLOT 1) ------------------ #
    # We'll restrict the visible x-axis to just around the Na window ± margin
    margin_fraction = 0.2
    na_plot_start_s = na_start_s - margin_fraction * na_delta_s
    na_plot_stop_s  = na_stop_s  + margin_fraction * na_delta_s

    for s in abf.sweepList:
        # Load the sweep for current (channel=0) and skip "blown" sweeps
        abf.setSweep(s, channel=0)
        if abs(abf.sweepY[-1]) > 1000:
            continue

        time = abf.sweepX
        current = abf.sweepY

        # Identify the analysis region for Na
        na_mask = (time >= na_start_s) & (time <= na_stop_s)
        if not np.any(na_mask):
            continue

        I_analysis = current[na_mask]
        t_analysis = time[na_mask]

        # Baseline is median within that window (for peak detection),
        # but we will report and plot raw peaks (no baseline subtraction).
        baseline = np.median(I_analysis)
        delta_i = I_analysis - baseline
        peak_idx = np.argmax(np.abs(delta_i))
        # raw peak = baseline + largest dev
        raw_peak_value = baseline + delta_i[peak_idx]
        raw_peak_time = t_analysis[peak_idx]

        # Now get the command voltage
        abf.setSweep(s, channel=1)
        v_hold = np.median(abf.sweepY)
        abf.setSweep(s, channel=0)
        cmd_trace = abf.sweepC + v_hold
        cmd_mask = na_mask
        command_v = np.median(cmd_trace[cmd_mask])
        command_v = round(command_v / 10) * 10  # round to nearest 10 mV if desired

        na_peaks.append(raw_peak_value)
        na_voltages.append(command_v)

        # -------------- PLOTTING --------------
        if ax_na is not None:
            # Plot only the window ± margin for Na
            na_plot_mask = (time >= na_plot_start_s) & (time <= na_plot_stop_s)
            ax_na.plot(time[na_plot_mask]*1000, current[na_plot_mask], 'k')
            # Mark the raw peak on the trace
            ax_na.scatter(raw_peak_time*1000, raw_peak_value, color='m',  zorder=5)

    # Set up the NA subplot
    if ax_na is not None:
        ax_na.set_xlim([na_plot_start_s*1000, na_plot_stop_s*1000])
        ax_na.set_xlabel("Time (ms)")
        ax_na.set_ylabel("I (pA)")
        ax_na.set_title(f"Na Window\n({Na_window[0]}-{Na_window[1]} ms)")

    # ----------------- K MEASUREMENT & PLOT (SUBPLOT 2) ------------------ #
    # For K, measure the mean in K_window but plot the entire sweep
    for s in abf.sweepList:
        abf.setSweep(s, channel=0)
        if abs(abf.sweepY[-1]) > 1000:
            continue

        time = abf.sweepX
        current = abf.sweepY

        # entire-sweep mask for plotting
        # (time >= 0) & (time <= time[-1]) would also be fine, but we'll just do everything
        full_mask = np.ones_like(time, dtype=bool)

        # measurement mask for K
        k_mask = (time >= k_start_s) & (time <= k_stop_s)
        if not np.any(k_mask):
            continue

        I_analysis = current[k_mask]
        I_mean = np.median(I_analysis)  # raw mean for K

        # command voltage
        abf.setSweep(s, channel=1)
        v_hold = np.median(abf.sweepY)
        abf.setSweep(s, channel=0)
        cmd_trace = abf.sweepC + v_hold
        command_v = np.median(cmd_trace[k_mask])
        command_v = round(command_v / 10) * 10

        k_means.append(I_mean)
        k_voltages.append(command_v)

        # -------------- PLOTTING --------------
        if ax_k is not None:
            # plot entire sweep
            ax_k.plot(time[full_mask]*1000, current[full_mask], 'k')
            # scatter the measured mean at the window's midpoint
            k_mid_t = (k_start_s + k_stop_s)/2
            ax_k.scatter(k_mid_t*1000, I_mean, color='c',  zorder=5)

    # Set up the K subplot
    if ax_k is not None:
        ax_k.set_xlim([time[0]*1000, time[-1]*1000])  # entire sweep
        ax_k.set_xlabel("Time (ms)")
        ax_k.set_ylabel("I (pA)")
        ax_k.set_title(f"K Window\n({K_window[0]}-{K_window[1]} ms)")

    # --------------------- BUILD RESULTS DICT (raw values) --------------------- #
    results = {}
    # Na
    key_na = f"Na_{Na_window}"
    results[key_na] = {}
    for v, peak in zip(na_voltages, na_peaks):
        results[key_na][v] = peak

    # K
    key_k = f"K_{K_window}"
    results[key_k] = {}
    for v, mean_i in zip(k_voltages, k_means):
        results[key_k][v] = mean_i

    # --------------------- I–V PLOT (SUBPLOT 3) --------------------- #
    if to_plot and ax_iv is not None:
        ax_iv.set_title("I–V")
        ax_iv.set_xlabel("Voltage (mV)")
        ax_iv.set_ylabel("I (pA)")

        # Sort by voltage for a cleaner line
        unique_na_voltages = sorted(set(na_voltages))
        unique_k_voltages  = sorted(set(k_voltages))

        # Extract in sorted order
        sorted_na_peaks = [results[key_na][v] for v in unique_na_voltages]
        sorted_k_means  = [results[key_k][v]   for v in unique_k_voltages]

        # Plot them
        ax_iv.plot(unique_na_voltages, sorted_na_peaks, '-o', color='m')
        ax_iv.plot(unique_k_voltages,  sorted_k_means,  '-o', color='c')

        # Add lines at x=0 and y=0
        ax_iv.axhline(0, color='k', linewidth=.25)
        ax_iv.axvline(0, color='k', linewidth=.25)

        # Remove legend if any
        ax_iv.get_legend_handles_labels()  # if you had a legend, remove it
        # No explicit call to legend -> ensures none is shown

    # ----------- Adjust layout, Save, and Return ----------- #
    if to_plot:
        plt.tight_layout()  # ensure everything fits
        os.makedirs('Saved_Figs/IV_Curves/', exist_ok=True)
        save_name = f"Saved_Figs/IV_Curves/IV_Curves_{abf.abfID}.{figopt['type']}"
        plt.savefig(save_name, dpi=figopt['dpi'])
        plt.show()

    return results
